In [1]:
!pwd

/Users/drbh/speaker_detection


# Run program from directory

In [2]:
from collections import defaultdict
from scipy.io import wavfile
import numpy as np
import cPickle as pickle
import traceback as tb
import itertools
import glob
import time
import os
import sys

# add to run in ipython
sys.path.append('./gui/')

from interface import ModelInterface
from utils import read_wav
from filters.silence import remove_silence

No module named ap


In [165]:
def task_enroll(model, input_dirs):
    m = model
    input_dirs = [os.path.expanduser(k) for k in input_dirs.strip().split()]
    dirs = itertools.chain(*(glob.glob(d) for d in input_dirs))
    dirs = [d for d in dirs if os.path.isdir(d)]
    files = []
    print(dirs)
    if len(dirs) == 0:
        print "No valid directory found!"
        sys.exit(1)
    for d in dirs:
        label = os.path.basename(d.rstrip('/'))

        wavs = glob.glob(d + '/*.wav')
        if len(wavs) == 0:
            print "No wav file found in {0}".format(d)
            continue
        print "Label {0} has files {1}".format(label, ','.join(wavs))
        for wav in wavs:
            fs, signal = read_wav(wav)
            
            new_signal = m.filter(signal)
            print("")
            print(len(new_signal))
            print("")
            print "After removed: {0} -> {1}".format(len(signal[1]), len(new_signal))
            print "Enroll: {:.4f} seconds".format(float(len(new_signal)) / fs)
            if len(new_signal) == 0:
                print "Error! Input is silent! Please enroll again"
            m.enroll(label, fs, new_signal)
    m.train()
    
    
def task_predict(input_files, input_model):
    m = ModelInterface.load(input_model)
    for f in glob.glob(os.path.expanduser(input_files)):
        fs, signal = read_wav(f)
        label = m.predict(fs, signal)
        print f, '->', label

##### TO DO

    gmmset = gui/gmmset.py
    skgmm  = gui/skgmm.py
    


# Test functionality

In [166]:
task_enroll("./People/David","model.out")

[]
No valid directory found!


SystemExit: 1

In [5]:
task_predict("/Users/drbh/Desktop/DV/02_my_name.wav","model.out")

/Users/drbh/Desktop/DV/02_my_name.wav -> DV


# In realtime

In [6]:
import pyaudio
import wave

In [109]:
console = True
def record_audio(WAVE_OUTPUT_FILENAME):
# 	direc = "/Users/drbh/speaker_detection/People/David/"
# 	curtime = time.strftime("%Y%m%d-%H%M%S")  
	CHUNK = 1024
	FORMAT = pyaudio.paInt16
	CHANNELS = 2
	RATE = 44100
	RECORD_SECONDS = 2
    
# 	WAVE_OUTPUT_FILENAME = direc + curtime + "apt.wav"

	WAVE_OUTPUT_FILENAME = WAVE_OUTPUT_FILENAME #"apt.wav"

	p = pyaudio.PyAudio()

	stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

	if console:
		print "* Recording audio..."

	frames = []

	for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
		data = stream.read(CHUNK)
		frames.append(data)

	if console:
		print "* done\n" 

	stream.stop_stream()
	stream.close()
	p.terminate()

	wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
	wf.setnchannels(CHANNELS)
	wf.setsampwidth(p.get_sample_size(FORMAT))
	wf.setframerate(RATE)
	wf.writeframes(b''.join(frames))
	wf.close()
    
def add_user(name):
    record_samples(name,1)
    fname = "/Users/drbh/speaker_detection/People/" + name + "/"
    print(fname)
    task_enroll(fname,"model.out")
    
    
def find_user():
    record_audio("./tmp.wav")
    task_predict("./tmp.wav","model.out")

In [110]:
def record_samples(name, n = 5):
    for i in range(1,n):
        dirc = "/Users/drbh/speaker_detection/People/" + name + "/"
        fname = time.strftime("%Y%m%d-%H%M%S") 
        print(i)
        record_audio( dirc + fname + ".wav")
    print("done")

In [ ]:
Model

In [114]:
# add_user("Kathy")

done
/Users/drbh/speaker_detection/People/Kathy/
['/Users/drbh/speaker_detection/People/Kathy/']
Label Kathy has files /Users/drbh/speaker_detection/People/Kathy/20161228-114409apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114414apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114419apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114424apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114429apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114435apt.wav
Start training...
0.319418907166  seconds


In [156]:
mods = ModelInterface()

In [ ]:
new_signal = self.backend.filter(*self.enrollWav)
        print "After removed: {0} -> {1}".format(len(self.enrollWav[1]), len(new_signal))
        print "Enroll: {:.4f} seconds".format(float(len(new_signal)) / Main.FS)
        if len(new_signal) == 0:
            print "Error! Input is silent! Please enroll again"
            return
        self.backend.enroll(name, Main.FS, new_signal)
        


In [162]:
mods.filter()

TypeError: enroll() takes exactly 4 arguments (1 given)

In [167]:
def add_user_to_model(model,name):
    m = model
    fname = "/Users/drbh/speaker_detection/People/" + name + "/"
    input_dirs = fname
    input_dirs = [os.path.expanduser(k) for k in input_dirs.strip().split()]
    dirs = itertools.chain(*(glob.glob(d) for d in input_dirs))
    dirs = [d for d in dirs if os.path.isdir(d)]
    files = []
    print(dirs)
    if len(dirs) == 0:
        print "No valid directory found!"
        sys.exit(1)
    for d in dirs:
        label = os.path.basename(d.rstrip('/'))

        wavs = glob.glob(d + '/*.wav')
        if len(wavs) == 0:
            print "No wav file found in {0}".format(d)
            continue
        print "Label {0} has files {1}".format(label, ','.join(wavs))
        for wav in wavs:
            fs, signal = read_wav(wav)
            m.enroll(label, fs, signal)
            
    m.train()

In [159]:
for name in ["David","Kathy"]:
    add_user_to_model(mods,name)

['/Users/drbh/speaker_detection/People/David/']
Label David has files /Users/drbh/speaker_detection/People/David/20161228-114456apt.wav,/Users/drbh/speaker_detection/People/David/20161228-114502apt.wav,/Users/drbh/speaker_detection/People/David/20161228-114507apt.wav,/Users/drbh/speaker_detection/People/David/20161228-114512apt.wav,/Users/drbh/speaker_detection/People/David/20161228-114517apt.wav,/Users/drbh/speaker_detection/People/David/20161228-114614apt.wav
Start training...
0.891692876816  seconds
['/Users/drbh/speaker_detection/People/Kathy/']
Label Kathy has files /Users/drbh/speaker_detection/People/Kathy/20161228-114409apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114414apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114419apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114424apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114429apt.wav,/Users/drbh/speaker_detection/People/Kathy/20161228-114435apt.wav
Start training...
1.541040

In [160]:
for count, (key, value) in enumerate(mods.features.iteritems(), 1):
    print count, key, len(value)

1 Kathy 1866
2 David 1866


In [155]:
mods.dump("model_two.out")

In [161]:
_tmp = ModelInterface().load("model_two.out")
for count, (key, value) in enumerate(_tmp.features.iteritems(), 1):
    print count, key, len(value)

1 Kathy 3732
2 David 3732


In [164]:
live = True
while live is True:
    for i in range(1,4):
        find_user()
    live = False

* Recording audio...
* done

./tmp.wav -> David
* Recording audio...
* done

./tmp.wav -> David
* Recording audio...
* done

./tmp.wav -> David


In [44]:
conv_result_list = []

* Recording audio...
* done

apt.wav -> David


In [45]:
task_predict("apt.wav","model.out")

apt.wav -> David


In [127]:
task_predict("./People/Kathy/20161228-114429apt.wav","model.out")

./People/Kathy/20161228-114429apt.wav -> Kathy


In [ ]:
CONV_INTERVAL = 0.4
CONV_DURATION = 1.5
CONV_FILTER_DURATION = CONV_DURATION
FS = 8000
TEST_DURATION = 3